## Prepare Environment

In [1]:
# !pip3 install pymongo
# !pip3 install mongoengine
# !pip3 install Faker
# !pip3 install mongomock

In [2]:
# %load_ext nb_black
# %load_ext autoreload
# %autoreload 2

In [3]:
# Prepare environment for importing from src
import sys
import os

sys.path.insert(0, "..")

## Import Dependencies 

In [4]:
import random
import datetime

from mongoengine import connect, get_connection

from src.data import initialize_db
from src.utils import drop_db

## Connect to Mock DB

In [5]:
from pymongo import MongoClient

client = connect("assignment", host="mongodb://127.0.0.1:27017")

In [97]:
if not os.environ.get("TEST"):
    drop_db(client, "assignment")

## Generate Fake Data & Insert Them to DB

In [8]:
if not os.environ.get("TEST"):
    initialize_db()

In [9]:
print(client.assignment.list_collection_names())
print(client.assignment.patient.find_one())

['sale', 'patient', 'product', 'comment', 'drug', 'prescription', 'pharmacy', 'user', 'company', 'product_item', 'driver', 'doctor', 'contract']
{'_id': ObjectId('601aefddd97804f997cd3730'), 'national_id': '3389083863', 'first_name': 'Shane', 'last_name': 'Ramirez', 'address': '65423 Rogers Common Apt. 594\nBlairmouth, ME 72858', 'birthdate': datetime.datetime(1978, 3, 4, 0, 0), 'password': 'raymond57', 'doctor_id': ObjectId('601aefddd97804f997cd372f')}


## Examples

In [10]:
print(client.assignment.drug.find_one({"formula": "CH3COOH"}))

{'_id': ObjectId('601aefdfd97804f997cd3ac3'), 'name': 'Midodrine Hydrochloride', 'company_id': ObjectId('601aefddd97804f997cd37fa'), 'formula': 'CH3COOH'}


In [11]:
list(
    client.assignment.drug.aggregate(
        [{"$group": {"_id": "$formula", "count": {"$sum": 1}}}]
    )
)

[{'_id': 'NaNO3', 'count': 13},
 {'_id': 'C5H8NO4Na', 'count': 3},
 {'_id': 'CH4N2S', 'count': 2},
 {'_id': 'MnSO4', 'count': 2},
 {'_id': 'CH3COOH', 'count': 4},
 {'_id': 'H2C2O4', 'count': 2},
 {'_id': 'CH3CHOHCH3', 'count': 2},
 {'_id': 'NH4I', 'count': 3},
 {'_id': 'Na2HPO4', 'count': 4},
 {'_id': 'C18H36O2', 'count': 4},
 {'_id': 'CaC2', 'count': 3},
 {'_id': 'C4H10', 'count': 4},
 {'_id': 'Na3AsO4', 'count': 2},
 {'_id': 'H2SO4', 'count': 10},
 {'_id': 'Na3N', 'count': 5},
 {'_id': 'CH3COONH4', 'count': 1},
 {'_id': 'C2H3ClO2', 'count': 1},
 {'_id': 'CH3CO2K', 'count': 4},
 {'_id': 'C2H4', 'count': 2},
 {'_id': 'C7H6O2', 'count': 4},
 {'_id': 'KF', 'count': 2},
 {'_id': 'C2H3N', 'count': 4},
 {'_id': 'PCl5', 'count': 6},
 {'_id': 'ZnSO4', 'count': 3},
 {'_id': 'Ag2O', 'count': 5},
 {'_id': 'SnCl2', 'count': 4},
 {'_id': 'KCN', 'count': 4},
 {'_id': 'LiBr', 'count': 5},
 {'_id': 'C4H8O2', 'count': 6},
 {'_id': 'CH3CH2OH', 'count': 2},
 {'_id': 'NaBr', 'count': 4},
 {'_id': 'NaClO'

In [12]:
client.assignment.patient.aggregate(
    [
        {
            "$lookup": {
                "from": "doctor",
                "localField": "doctor_id",
                "foreignField": "_id",
                "as": "doctor",
            }
        },
        {"$match": {"doctor.first_name": "Robert"}},
        {"$count": "patients"},
    ]
).next()

{'patients': 9}

## Query Assignments

In [6]:
# نام داروخانه هایی که شماره تلفن آنها با 1+ شروع می شود
print("##1##")
a1 = list(
    client.assignment.pharmacy.find(
        filter={
            'telephone': {
                '$regex': r'^\+1.+$'
            }
        },
        projection={"name": 1, "_id": 0},
    )
)
print(a1)

##1##
[{'name': 'Rogers-Cardenas'}, {'name': 'Jones, Fox and Romero'}, {'name': 'Lee, Moore and Ferrell'}, {'name': 'Ellison-Arias'}, {'name': 'Henry Inc'}, {'name': 'Lopez-Buchanan'}, {'name': 'Weber-Zamora'}, {'name': 'Kennedy Inc'}, {'name': 'Ramirez Group'}, {'name': 'Fry-Douglas'}, {'name': 'Smith, Kemp and Hughes'}, {'name': 'Nicholson, Smith and Hernandez'}]


In [7]:
# شماره ملی افرادی که بعد از تاریخ datetime.datetime(2000, 1, 1, 0, 0) متولد شده اند
print("##2##")
a2 = list(
    client.assignment.patient.find(
        filter={
            'birthdate': {
                '$gt': datetime.datetime(2000, 1, 1, 0, 0)
            }
        },
        projection={"national_id": 1, "_id": 0},
    )
)
print(a2)

##2##
[{'national_id': '7510799118'}, {'national_id': '8069901627'}, {'national_id': '7059640165'}, {'national_id': '9599534283'}, {'national_id': '5973832060'}, {'national_id': '5899441715'}, {'national_id': '9812387800'}, {'national_id': '4271739627'}, {'national_id': '2091241862'}, {'national_id': '0249724399'}, {'national_id': '0242403381'}, {'national_id': '9074260880'}, {'national_id': '3578243477'}, {'national_id': '0961701595'}, {'national_id': '8114578870'}, {'national_id': '8816502544'}, {'national_id': '1123274897'}, {'national_id': '4815269255'}, {'national_id': '8519537749'}, {'national_id': '5700412918'}, {'national_id': '2546862728'}, {'national_id': '8864078802'}, {'national_id': '0186074290'}, {'national_id': '2365401814'}]


In [8]:
# تعداد نسخه هایی که دارای حداقل 15 دارو هستند
print("##3##")
a3 = client.assignment.prescription.find(
    filter={
        '$where': "this.items.length > 15"
    }
).count()
print(a3)

##3##
20


In [9]:
# کد ملی بیمارانی که اسم پزشک آنها "Robert" است
print("##4##")
a4 = list(
    client.assignment.patient.aggregate(
        [
            { 
                "$lookup": {
                    'from': 'doctor',
                    'localField': 'doctor_id',
                    'foreignField': '_id',
                    'as': 'doctor'
                }
            },
            {
                "$unwind": "$doctor"
            },
            {
                "$match": {
                    'doctor.first_name': 'Robert'
                }
            },
            {
                '$project': {
                    'national_id': 1,
                     "_id": 0,
                }
            }
        ]
    )
)
print(a4)

##4##
[{'national_id': '3389083863'}, {'national_id': '0405411526'}, {'national_id': '9267389975'}, {'national_id': '0242403381'}, {'national_id': '0961701595'}, {'national_id': '2611231261'}, {'national_id': '3153888245'}, {'national_id': '4991004202'}, {'national_id': '0259364502'}]


In [10]:
# نام داروخانه ای که دارویی به گرانترین قیمت به آن فروخته شده است
print("##5##")
a5 = client.assignment.sale.aggregate(
    [
        {
            '$group': {
                '_id': None,
                "maxPrice": {
                    "$max": "$price"
                }
            }
        },
        { 
            "$lookup": {
                'from': 'sale',
                'localField': 'maxPrice',
                'foreignField': 'price',
                'as': 'sale'
            }
        },
        {
            "$unwind": "$sale"
        },
        { 
            "$lookup": {
                'from': 'pharmacy',
                'localField': 'sale.pharmacy_id',
                'foreignField': '_id',
                'as': 'pharmacy'
            }
        },
        {
            "$unwind": "$pharmacy"
        },
        {
            '$project': {
                'pharmacy.name': 1,
                "_id": 0,
            }
        },
        {
            "$replaceRoot": {
                "newRoot": "$pharmacy"
            }
        }
    ]
).next()
print(a5)

##5##
{'name': 'Clarke and Sons'}


In [11]:
# نام و فرمول پنج دارویی که گران ترین قیمت برای آنها ثبت شده است
print("##6##")
a6 = list(
    client.assignment.sale.aggregate(
        [
            {
                "$group": {
                    '_id': '$drug_id',
                    'max_price': {
                        '$max': '$price'
                    }
                }
            },
            {
                '$sort': {
                    'max_price': -1
                }
            },
            {
                '$limit': 5
            },
            {
                '$lookup': {
                    'from': 'drug',
                    'localField': '_id',
                    'foreignField': '_id',
                    'as': 'drug'
                }
            },
            {
                "$unwind": "$drug"
            },
            {
                '$project': {
                    'drug.name': 1,
                    'drug.formula': 1,
                    "_id": 0,
                }
            },
            {
                "$replaceRoot": {
                    "newRoot": "$drug"
                }
            }
        ]
    )
)
print(a6)

##6##
[{'name': 'Cyanocobalamin', 'formula': 'H2O'}, {'name': 'Trihexyphenidyl', 'formula': 'C4H8O2'}, {'name': 'Phenytoin', 'formula': 'Al'}, {'name': 'Estradiol', 'formula': 'C3H7NO2'}, {'name': 'Florinef', 'formula': 'KF'}]


In [39]:
# نام تمام داروهایی که در تاریخ datetime.datetime(2020, 9, 23, 0, 0) تجویز شده اند
print("##7##")
a7 = list(
    client.assignment.prescription.aggregate(
        [
            {
                '$match': {
                    'date': datetime.datetime(2020, 9, 23, 0, 0)
                }
            },
            {
                '$project': {
                    '_id': 0,
                    'items.drug_id': 1
                }
            },
            {
                "$replaceRoot": {
                    "newRoot": {
                        "$mergeObjects":[
                            {
                                '_id': '$items.drug_id'
                            }
                        ]
                    }
                }
            },
            {
                "$unwind": "$_id"
            },
            {
                '$lookup': {
                    'from': 'drug',
                    'localField': '_id',
                    'foreignField': '_id',
                    'as': 'drug'
                }
            },
            {
                "$unwind": "$drug"
            },
            {
                '$replaceRoot': {
                    'newRoot': '$drug'
                }
            },
            {
                "$sort" : {
                    "company_id" : 1
                }
            },
            {
                '$project': {
                    '_id': 0,
                    'name': 1
                }
            },
        ]
    )
)
print(a7)

##7##
[{'name': 'Oxycodone HCl'}, {'name': 'Demadex'}, {'name': 'Ceftin'}, {'name': 'Etrafon'}]


In [13]:
# نام تمام کارخانه هایی که داروی با فرمول "C2H6Na4O12" را تولید می کنند
print("##8##")
a8 = list(
    client.assignment.drug.aggregate(
        [
            {
                '$match': {
                    'formula': 'C2H6Na4O12'
                }
            },
            {
                '$lookup': {
                    'from': 'company',
                    'localField': 'company_id',
                    'foreignField': '_id',
                    'as': 'company'
                }
            },
            {
                "$unwind": "$company"
            },
            {
                '$replaceRoot': {
                    'newRoot': '$company'
                }
            },
            {
                '$project': {
                    '_id': 0,
                    'name': 1
                }
            },
        ]
    )
)
print(a8)

##8##
[{'name': 'Williams, Parker and Cross'}, {'name': 'Walker Ltd'}, {'name': 'Moore, Davis and Rodriguez'}, {'name': 'Lopez-Johnson'}, {'name': 'Davis Group'}, {'name': 'Brock-White'}]


In [14]:
# کاربرانی که در سبد آنها ده BasketItem وجود دارد
print("##9##")
a9 = list(
    client.assignment.user.find(
        filter={
            '$where': 'this.basket.length == 10'
        },
        projection={"email": 1, "_id": 0},
    )
)
print(a9)

##9##
[{'email': 'walkerlindsey@hotmail.com'}, {'email': 'terriramirez@yahoo.com'}, {'email': 'lawrence49@gmail.com'}, {'email': 'jonesdeborah@hotmail.com'}, {'email': 'kyle88@hotmail.com'}, {'email': 'wangdavid@yahoo.com'}, {'email': 'osimmons@yahoo.com'}, {'email': 'coxkarl@yahoo.com'}, {'email': 'williamsmichelle@gmail.com'}, {'email': 'castrokaitlin@gmail.com'}, {'email': 'hernandezkatherine@yahoo.com'}, {'email': 'carolmoore@hotmail.com'}, {'email': 'kathleen67@yahoo.com'}, {'email': 'hornejasmine@hotmail.com'}]


In [19]:
print("##10##")
a10 = client.assignment.product_item.aggregate(
    [
        {
            '$match': {
                'size': 'XL'
            }
        },
        {
            '$group': {
                '_id': None,
                'sum': {
                    '$sum': '$quantity'
                }
            }
        },
        {
            '$project': {
                '_id': 0,
                'sum': 1
            }
        }
    ]
).next()
print(a10)

##10##
{'sum': 15593}


In [22]:
# شماره ملی رانندگانی که پلاک آنها به 25 ختم می شود
print("##11##")
a11 = list(
    client.assignment.driver.find(
        filter={
            'license_plate': {
                '$regex': r'^.*25$'
            }
        },
        projection={"_id": 0, "national_id": 1},
    )
)
print(a11)

##11##
[{'national_id': '1956964607'}, {'national_id': '8157341591'}]


In [24]:
print("##12##")
a12 = list(
    client.assignment.comment.find(
        filter={
            'text': {
                '$regex': r'(great|good)'
            }
        },
        projection={"_id": 0, "text": 1},
    )
)
print(a12)

##12##
[{'text': 'Church certainly question share that job ever capital great what mean continue.'}, {'text': 'Piece good reason too evening probably low pull political.'}]


In [31]:
print("##13##")
a13 = client.assignment.comment.aggregate(
    [
        {
            '$match': {
                'rating': 5
            }
        },
        {
            '$count': 'count'
        }
    ]
).next()
print(a13)

##13##
{'count': 12}


In [40]:
answers = {
    "a1": a1,
    "a2": a2,
    "a3": a3,
    "a4": a4,
    "a5": a5,
    "a6": a6,
    "a7": a7,
    "a8": a8,
    "a9": a9,
    "a10": a10,
    "a11": a11,
    "a12": a12,
    "a13": a13,
}

In [41]:
import json
with open("answers.json", "r") as json_file:
    target = json.load(json_file)

In [42]:
correct = 0
for i in range(1, 14):
    if answers["a{}".format(i)] == target["a{}".format(i)]:
        print("Query {:2d} Correct!".format(i))
        correct += 1
    else:
        print("Query {:2d} Wrong!".format(i))
print(correct)

Query  1 Correct!
Query  2 Correct!
Query  3 Correct!
Query  4 Correct!
Query  5 Correct!
Query  6 Correct!
Query  7 Correct!
Query  8 Correct!
Query  9 Correct!
Query 10 Correct!
Query 11 Correct!
Query 12 Correct!
Query 13 Correct!
13


## Print Result to File  

In [43]:
# Set your student number
student_number = 90000000
file_path = os.path.join(
    os.getenv("OUTPUT_DIR", "."), "{}.json".format(student_number)
)
with open(file_path, "w") as file:
    corrects = []
    wrongs = []
    for i in range(1, 14):
        if answers["a{}".format(i)] == target["a{}".format(i)]:
            corrects.append(i)
        else:
            wrongs.append(i)
    json.dump({"corrects": corrects, "wrongs": wrongs, "score": len(corrects)}, file)